<a href="https://colab.research.google.com/github/JuanArmas/BigData7RO/blob/main/Algoritmo_genetico_evolutivo_JuanArmas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Problema del Viajante de comercios, por Juan Armas Alemán:

Código basado en lo descrito en los siguientes videos:

https://www.youtube.com/watch?v=3Kzj2FNaua8

https://www.youtube.com/watch?v=L7ABeOYXPck&list=PL6VsnkqbwvkKtHPPpFTlDExnImvQ6T0MR&index=2

In [ ]:
import numpy as np
import random


matriz de distancia

In [ ]:
# Definir las ciudades
ciudades = ["Las Palmas", "Telde", "Gáldar", "Vecindario", "Teror", "Santa Lucía", "Maspalomas", "Agaete"]

# Definir la matriz de distancias entre las ciudades (valores aproximados, reemplazar con valores reales)
matriz_distancias = [
    [0, 12.7, 25.5, 19.3, 15.4, 51.5, 41.2, 21.1],  # Distancias desde Las Palmas a otras ciudades
    [12.7, 0, 12.8, 7.6, 5.7, 38.7, 30.3, 18.8],    # Distancias desde Telde a otras ciudades
    [25.5, 12.8, 0, 12.1, 9.2, 40.3, 31.9, 20.4],   # Distancias desde Gáldar a otras ciudades
    [19.3, 7.6, 12.1, 0, 7.1, 36.1, 27.7, 16.2],    # Distancias desde Vecindario a otras ciudades
    [15.4, 5.7, 9.2, 7.1, 0, 31.2, 22.8, 11.3],     # Distancias desde Teror a otras ciudades
    [51.5, 38.7, 40.3, 36.1, 31.2, 0, 11.7, 38.6],  # Distancias desde Santa Lucía a otras ciudades
    [41.2, 30.3, 31.9, 27.7, 22.8, 11.7, 0, 27.5],  # Distancias desde Maspalomas a otras ciudades
    [21.1, 18.8, 20.4, 16.2, 11.3, 38.6, 27.5, 0]   # Distancias desde Agaete a otras ciudades
]

Definimos los hiperpárametros

In [ ]:
# Hiperparámetros
tam_poblac = 8  # Tamaño de la población (cantidad de individuos)
tasa_mut = 0.2  # Tasa de mutación (probabilidad de mutar un individuo)
tasa_recomb = 2  # Tasa de recombinación (proporción de individuos que serán recombinados)
max_iterac = 100  # Número máximo de iteraciones o épocas
umbral_fitness = 9.5  # Umbral de fitness para considerar que es aceptable
max_iterac_sin_mejora = 5  # Número de iteraciones sin mejora para detener el algoritmo si no hay mejoras


Generacion de un individuo aleatorio con longitud poblacional:

- Será el punto de partida y la primera propuesta de ruta óptima sobre la que trabajará el algoritmo.

In [ ]:
def generar_individuo():
    individuo = list(range(len(ciudades)))
    random.shuffle(individuo)
    return individuo

Funcion Recombinacion.

- Se crean dos hijos usando a dos individuos Padres, con el fin de que los valores de uno sirvan como indice del otro, alternandose entre ellos como indice para generar de esta manera a ambos hijos resultantes.
-

In [ ]:
def recombinar(padre1, padre2):
    hijo = [padre1[padre2[i]] for i in range(len(padre1))]
    return hijo

Funcion mutacion:
- Si el número aleatorio generado es menor que el hiperparámetro, tasa_mut, se procede a la mutación. De lo contrario, el individuo permanece sin cambios. Nos aseguramos así que no se apliquen mutaciones en todos los individuos creados.

In [ ]:
def mutar_individuo(individuo):
    if random.random() < tasa_mut: #genera un num entre 0-1 y compara con el hiperparámetro
        # Selecciona aleatoriamente dos índices distintos en el individuo para intercambiar
        id1, id2 = random.sample(range(len(individuo)), 2)
        individuo[id1], individuo[id2] = individuo[id2], individuo[id1] #se intercambian
    return individuo #devuelve individuo mutado

Funcion fitness
- Calcula la distancia total recorrida por el individuo en la matriz de distancias según el orden establecido,

sumando cada par de ciudades consecutivas y añadiendo esta suma a la distancia total, para evaluar más adelante, el recorrido minimo y máximo


In [ ]:
def fitness(individuo, matriz_distancias):
    distancia_total = 0
    for i in range(len(individuo) - 1):
        ciudad_actual = individuo[i]
        siguiente_ciudad = individuo[i + 1]# Obtiene la siguiente ciudad en el recorrido del individuo
        #suma la distancia entre dos ciudades consecutivas en el recorrido indicado
        distancia_total += matriz_distancias[ciudad_actual][siguiente_ciudad]
    # Suma la distancia desde la última ciudad hasta la inicial
    distancia_total += matriz_distancias[individuo[-1]][individuo[0]]
    return distancia_total

Funcion de ejecución de iteraciones:

Se ejecuta la carga de iteraciones para las evoluciones, creación de población de indivíduos y para hallar la mejor ruta en base a la distancia recorrida (fitness). Buscando la mejor solución o saliendo en caso de alcanzar la condicion de salida (max_iterac_sin_mejora)

In [ ]:
def ejecutar_iteraciones(poblacion, mejor_fitness_global, mejor_ruta_global):
    iter_sin_mejora = 0

    for iteracion in range(max_iterac):
        #calcula la suma de distancia de cada individuo de la poblacion
        fitness_poblacion = [fitness(individuo, matriz_distancias) for individuo in poblacion]
        mejor_fitness_id = fitness_poblacion.index(min(fitness_poblacion))#indice del mejor fitness
        #almacena el mejor fitness e individuo
        mejor_fitness = fitness_poblacion[mejor_fitness_id]
        mejor_individuo = poblacion[mejor_fitness_id]

        print(f"Iteración {iteracion + 1} - Suma de distancias: {mejor_fitness}")
        #condicional actualiza la ruta y el fitness cuando encuentra uno mejor.
        if mejor_fitness < mejor_fitness_global:
            mejor_fitness_global = mejor_fitness
            mejor_ruta_global = mejor_individuo

        for i in range(len(poblacion)):#mutar poblacion
            poblacion[i] = mutar_individuo(poblacion[i])

        #recombinación de indivíduos
        num_recombinados = int(tasa_recomb * tam_poblac)
        for _ in range(num_recombinados):
            padre1, padre2 = random.sample(poblacion, 2)
            hijo = recombinar(padre1, padre2)
            poblacion.append(hijo)

        #mantiene el tamaño de la poblacion constante
        num_nuevos = tam_poblac - len(poblacion)
        nuevos_individuos = [generar_individuo() for _ in range(num_nuevos)]
        poblacion.extend(nuevos_individuos)# añadiendo individuos aleatorios

        if mejor_fitness <= umbral_fitness:
            break#condicion de parada en caso de alcanzar el hiperparametro umbralFitness

        # condicional para detener la ejecución si no se encuentra
        # mejora tras un num de iteraciones hiperparametradas
        if mejor_fitness == fitness_poblacion[0]:
            iter_sin_mejora += 1
        else:
            iter_sin_mejora = 0

        if iter_sin_mejora >= max_iterac_sin_mejora:
            break

    return poblacion, mejor_fitness_global, mejor_ruta_global

función genética:
Auna todas las funciones ejecutadas y muestra los resultados:



In [ ]:
def algoritmo_genetico():
    #creación de una población de individuos iniciales
    poblacion = [generar_individuo() for _ in range(tam_poblac)]

    #definición de variables
    mejor_fitness_global = float('inf')
    mejor_ruta_global = None

    #recogida de datos de la funcion de iteracion
    poblacion, mejor_fitness_global, mejor_ruta_global = ejecutar_iteraciones(poblacion,
                                                                              mejor_fitness_global,
                                                                              mejor_ruta_global)

    # Cálculo del mejor individuo y su fitness
    mejor_fitness = min([fitness(individuo, matriz_distancias) for individuo in poblacion])
    mejor_individuo = poblacion[[fitness(individuo,
                                         matriz_distancias) for individuo in poblacion].index(mejor_fitness)]

    #uso array de la mejor ruta como indice de la lista de ciudades
    mejor_ruta_nombres = [ciudades[id] for id in mejor_ruta_global]

    # Impresión de la mejor ruta y su fitness
    print("\nMejor orden de visitas a las ciudades:")
    for ciudad in mejor_ruta_nombres:
        print(ciudad)

    print("\nFitness de la mejor solución:", mejor_fitness)

    #devolución de la mejor solucion
    return mejor_ruta_nombres, mejor_fitness

#imprime las iteraciones y la mejor solucion
mejor_orden, mejor_fitness = algoritmo_genetico()

Iteración 1 - Suma de distancias: 153.0
Iteración 2 - Suma de distancias: 143.7
Iteración 3 - Suma de distancias: 139.59999999999997
Iteración 4 - Suma de distancias: 139.59999999999997
Iteración 5 - Suma de distancias: 139.59999999999997
Iteración 6 - Suma de distancias: 139.59999999999997
Iteración 7 - Suma de distancias: 140.6
Iteración 8 - Suma de distancias: 140.6
Iteración 9 - Suma de distancias: 140.6
Iteración 10 - Suma de distancias: 140.6
Iteración 11 - Suma de distancias: 140.6
Iteración 12 - Suma de distancias: 135.8
Iteración 13 - Suma de distancias: 139.60000000000002
Iteración 14 - Suma de distancias: 141.39999999999998
Iteración 15 - Suma de distancias: 138.79999999999998
Iteración 16 - Suma de distancias: 138.79999999999998
Iteración 17 - Suma de distancias: 138.79999999999998
Iteración 18 - Suma de distancias: 138.79999999999998
Iteración 19 - Suma de distancias: 140.9
Iteración 20 - Suma de distancias: 138.2
Iteración 21 - Suma de distancias: 138.2
Iteración 22 - Sum